## Установка и обновление пакетов для Telegram бота

In [ ]:
%pip install --quiet --upgrade telebot redis telegram python-telegram-bot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.4/545.4 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


## Библиотеки и модули для работы с Redis и Telegram ботом

In [ ]:
import redis
import telebot
import json
from telebot import types
from telebot.util import async_dec

## Telegram бот для создания моделей домов с использованием Redis

In [ ]:
TOKEN = '6266329963:AAFAo5KOFAjRwUNFUGaX-WchaRXA82CA7iI'
bot = telebot.TeleBot(TOKEN)

# Установка соединения с Redis
redis_host = 'redis-13865.c1.asia-northeast1-1.gce.cloud.redislabs.com'
redis_port = 13865
redis_password = 'AjuhoBzf8IF3TmxG1SE3yFoMWh2q19Kt'
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password)
r = redis.Redis(host=redis_host, port=redis_port, password=redis_password)

button_sets = [
    {
        "question": "Выберете бюджет:",
        "buttons": ["$", "$$", "$$$", "СБРОС и заново"]
    },
    {
        "question": "Выберите размер дома:",
        "buttons": ["Небольшой", "Средний", "Большой", "СБРОС и заново"]
    },
    {
        "question": "Выберите количество этажей:",
        "buttons": ["1", "2", "3", "Разноуровневый дом", "СБРОС и заново"]
    },
    {
        "question": "Выберете материал фасада:",
        "buttons": ["Комбинированный", "Дерево", "Штукатурка", "Кирпич", "СБРОС и заново"]
    },
    {
        "question": "Выберете цвет фасада:",
        "buttons": ["Белый", "Бежевый", "Серый", "Коричневый", "Мультицвет", "Черный", "Красный", "Желтый",
                    "СБРОС и заново"]
    },
    {
        "question": "Выберете форму крыши:",
        "buttons": ["Двускатная", "Вальмовая", "Односкатная", "Плоская", "Полувальмовая", "СБРОС и заново"]
    },
    {
        "question": "Выберете цвет крыши:",
        "buttons": ["Серый", "Черный", "Коричневый", "Красный", "СБРОС и заново"]
    },
]

# Сопоставление выбора пользователя с условием в модели дома
condition = {
    "$": "$",
    "$$": "$$",
    "$$$": "$$$",
    "Небольшой": "compact",
    "Средний": "medium",
    "Большой": "large",
    "1": "one-story",
    "2": "two-story",
    "3": "three-story",
    "Разноуровневый дом": "split-level",
    "Комбинированный": "mixed siding",
    "Дерево": "wood",
    "Штукатурка": "stucco",
    "Кирпич": "brick",
    "Белый": "white",
    "Бежевый": "beige",
    "Серый": "gray",
    "Коричневый": "brown",
    "Мультицвет": "multi",
    "Черный": "black",
    "Красный": "red",
    "Желтый": "yellow",
    "Двускатная": "gable",
    "Вальмовая": "hip",
    "Односкатная": "shed",
    "Плоская": "flat",
    "Полувальмовая": "clipped gable",
    "бюджет": "budget",
    "размер дома": "size",
    "количество этажей": "number of stories",
    "материал фасада": "siding material",
    "цвет фасада": "siding color",
    "форму крыши": "roof type",
    "цвет крыши": "roof color"
}

condition_users = []

user_choices = {}

@bot.message_handler(commands=['start'])
@async_dec()
def start(message):
    # Начало создания модели дома
    send_buttons(message.chat.id, 0)


@bot.message_handler(func=lambda message: True)
@async_dec()
def handle_message(message):
    chat_id = message.chat.id
    user_id = str(chat_id)

    if message.text != "СБРОС и заново":
        # Обработка выбора пользователя

        if user_id not in user_choices:
            user_choices[user_id] = []

        if user_id not in condition_users:
            # Добавление выбора пользователя и проверка завершения формы

            choice = message.text
            choice = condition[choice]
            user_choices[user_id].append(choice)

            if len(user_choices[user_id]) == len(button_sets):
                # Все параметры выбраны, сохранение данных в Redis и отправка сообщения пользователю

                json_data = {
                    "id": user_id,
                }

                choices = user_choices[user_id]

                for item in range(len(choices)):
                    data_for_dict = button_sets[item]['question'][9:-1]
                    json_data[condition[data_for_dict]] = choices[item]

                json_data['image'] = ''

                json_file = json.dumps(json_data, ensure_ascii=False)

                save_json_file(json_file)

                redis_key = user_id
                redis_client.set(redis_key, json_file)

                # Проверка успешности сохранения в Redis
                result = redis_client.get(redis_key)

                if result:
                    print('Файл JSON успешно сохранен в Redis.')
                else:
                    print('Не удалось сохранить файл JSON в Redis.')

                json_data['condition_user'] = '1'

                condition_users.append(user_id)
                bot.send_message(chat_id=user_id,
                                 text="Вы отправили запрос на создание модели дома. Среднее время ожидания: 1 минута")

                user_choices[user_id] = []

            if user_id not in condition_users:
                send_buttons(chat_id, len(user_choices[user_id]))

        else:
            # Пользователь уже отправил запрос на создание модели дома
            bot.send_message(chat_id=user_id, text="Вы уже отправили запрос на создание модели дома. Ожидайте")

    else:
        # Сброс выбора и начало заново

        keys = r.keys("*")
        keys_str = []

        for byte_key in keys:
            str_var = byte_key.decode('utf-8')
            keys_str.append(str_var)

        if user_id not in condition_users:
            user_choices[user_id] = []
            send_buttons(chat_id, 0)

        elif (str(user_id in condition_users) == 'True') and (str(user_id not in keys_str) == 'True'):
            condition_users.remove(user_id)
            send_buttons(chat_id, 0)
        else:
            bot.send_message(chat_id=user_id, text="Вы уже отправили запрос на создание модели дома. Ожидайте")


@async_dec()
def send_buttons(chat_id, set_index):
    # Отправка пользователю кнопок для выбора
    set_data = button_sets[set_index]
    question = set_data["question"]
    buttons = set_data["buttons"]
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(*[types.KeyboardButton(button) for button in buttons])

    bot.send_message(chat_id, question, reply_markup=markup)


def save_json_file(json_data):
    # Сохранение JSON-файла
    print(json_data)


bot.polling()

{"id": "1655476081", "budget": "$$$", "size": "large", "number of stories": "three-story", "siding material": "mixed siding", "siding color": "multi", "roof type": "clipped gable", "roof color": "gray", "image": ""}
Файл JSON успешно сохранен в Redis.
